In [1]:
!pip install "thinc>=8.0.0a0" ml_datasets "tqdm>=4.41"

  Using cached ml_datasets-0.1.6.tar.gz (8.0 kB)
  Created wheel for ml-datasets: filename=ml_datasets-0.1.6-py3-none-any.whl size=11254 sha256=d5c509d212646b11ebebb0f15e75780c6638419ad7cba122b6a473e8e9e11023
  Stored in directory: /Users/jean.metz/Library/Caches/pip/wheels/34/ac/d7/5df981dd308bb698a0dab0e0c87321fa7d2863c867d9a852f5
Successfully built ml-datasets


In [2]:
from thinc.api import prefer_gpu
prefer_gpu()


False

In [ ]:
import ml_datasets
from tqdm.notebook import tqdm
from thinc.api import fix_random_seed

In [3]:
fix_random_seed(0)

def train_model(model, optimizer, n_iter, batch_size):
    (train_X, train_y), (dev_X, dev_y) = ml_datasets.ud_ancora_pos_tags()
    model.initialize(X=train_X[:5], Y=train_y[:5])
    for n in range(n_iter):
        loss = 0.0
        batches = model.ops.multibatch(batch_size, train_X, train_y, shuffle=True)
        for X, Y in tqdm(batches, leave=False):
            Yh, backprop = model.begin_update(X)
            d_loss = []
            for i in range(len(Yh)):
                d_loss.append(Yh[i] - Y[i])
                loss += ((Yh[i] - Y[i]) ** 2).sum()
            backprop(d_loss)
            model.finish_update(optimizer)
        score = evaluate(model, dev_X, dev_y, batch_size)
        print(f"{n}\t{loss:.2f}\t{score:.3f}")
        
def evaluate(model, dev_X, dev_Y, batch_size):
    correct = 0
    total = 0
    for X, Y in model.ops.multibatch(batch_size, dev_X, dev_Y):
        Yh = model.predict(X)
        for yh, y in zip(Yh, Y):
            correct += (y.argmax(axis=1) == yh.argmax(axis=1)).sum()
            total += y.shape[0]
    return float(correct / total)

NameError: name 'fix_random_seed' is not defined

## Composing the model in code

Here's the model definition, using the `>>` operator for the `chain` combinator.
The `strings2arrays` transform converts a sequence of strings to a list of arrays. `with_array` 
transforms sequences (the sequences of arrays) into a contiguous 2-dimensional array on the way into 
and out of the model it wraps. This means our model has the following signature: 
`Model[Sequence[str], Sequence[Array2d]]`.


In [ ]:
from thinc.api import Model, chain, strings2arrays, with_array, HashEmbed, expand_window, Relu, Softmax, Adam, warmup_linear

width = 32
vector_width = 16
nr_classes = 17
learn_rate = 0.001
n_iter = 10
batch_size = 128

with Model.define_operators({">>": chain}):
    model = strings2arrays() >> with_array(
        HashEmbed(nO=width, nV=vector_width, column=0)
        >> expand_window(window_size=1)
        >> Relu(nO=width, nI=width * 3)
        >> Relu(nO=width, nI=width)
        >> Softmax(nO=nr_classes, nI=width)
    )
optimizer = Adam(learn_rate)

## Composing the model via config file

If we want to rebuild the model defined above in a config file, we first need to break down its structure:

* `chain` (any number of positional arguments)
  * `strings2arrays` (no arguments)
  * `with_array` (one argument **layer**)
    * **layer:** `chain` (any number of positional arguments)
      * `HashEmbed`
      * `expand_window`
      * `Relu`
      * `Relu`
      * `Softmax`

`chain` takes a variable number of positional arguments (the layers to compose). 
In the config, positional arguments can be expressed using `*` in the dot notation. For example, 
`model.layer` could describe a function passed to `model` as the argument `layer`, while `model.*.relu` 
defines a positional argument passed to `model`. The name of the argument, e.g. `relu` – doesn't matter 
in this case. It just needs to be unique.

> ⚠️ **Important note:** It is recommended to use a hybrid approach: wrap the model definition in a registered function
> and configure it via the config.

In [ ]:
CONFIG = """
[hyper_params]
width = 32
vector_width = 16
learn_rate = 0.001

[training]
n_iter = 10
batch_size = 128

[model]
@layers = "chain.v1"

[model.*.strings2arrays]
@layers = "strings2arrays.v1"

[model.*.with_array]
@layers = "with_array.v1"

[model.*.with_array.layer]
@layers = "chain.v1"

[model.*.with_array.layer.*.hashembed]
@layers = "HashEmbed.v1"
nO = ${hyper_params:width}
nV = ${hyper_params:vector_width}
column = 0

[model.*.with_array.layer.*.expand_window]
@layers = "expand_window.v1"
window_size = 1

[model.*.with_array.layer.*.relu1]
@layers = "Relu.v1"
nO = ${hyper_params:width}
nI = 96

[model.*.with_array.layer.*.relu2]
@layers = "Relu.v1"
nO = ${hyper_params:width}
nI = ${hyper_params:width}

[model.*.with_array.layer.*.softmax]
@layers = "Softmax.v1"
nO = 17
nI = ${hyper_params:width}

[optimizer]
@optimizers = "Adam.v1"
learn_rate = ${hyper_params:learn_rate}
"""


from thinc.api import registry, Config

config = Config().from_str(CONFIG)
loaded_config = registry.make_from_config(config)

model = loaded_config["model"]
optimizer = loaded_config["optimizer"]
n_iter = loaded_config["training"]["n_iter"]
batch_size = loaded_config["training"]["batch_size"]
train_model(model, optimizer, n_iter, batch_size)

## Composing the model with code and config


In [ ]:
import thinc
from thinc.api import Model, chain, strings2arrays, with_array, HashEmbed, expand_window, Relu, Softmax,\
    Adam, warmup_linear

@thinc.registry.layers("cnn_tagger.v1")
def create_cnn_tagger(width: int, vector_width: int, nr_classes: int = 17) -> Model:
    with Model.define_operators({">>": chain}):
        model = strings2arrays() >> with_array(
            HashEmbed(nO=width, nV=vector_width, column=0)
            >> expand_window(window_size=1)
            >> Relu(nO=width, nI=width * 3)
            >> Relu(nO=width, nI=width)
            >> Softmax(nO=nr_classes, nI=width)
        )
    return model

CONFIG = """
[hyper_params]
width = 32
vector_width = 16
learn_rate = 0.001

[training]
n_iter = 10
batch_size = 128

[model]
@layers = "cnn_tagger.v1"
width = ${hyper_params:width}
vector_width = ${hyper_params:vector_width}
nr_classes = 17

[optimizer]
@optimizers = "Adam.v1"
learn_rate = ${hyper_params:learn_rate}
"""

loaded_config = registry.make_from_config(Config().from_str(CONFIG))

model = loaded_config["model"]
optimizer = loaded_config["optimizer"]
n_iter = loaded_config["training"]["n_iter"]
batch_size = loaded_config["training"]["batch_size"]
train_model(model, optimizer, n_iter, batch_size)